# TP1.  Systèmes de recommandation


# SD-TSIA 211

## 1. Présentation du modèle
### Question 1.1

L'importation des données nécéssite une connexion internet et prend un peu de temps (~15sec)
Vous pourrez changer la valeur assignée à la variable "file_path" si vous disposez déjà du fichier u.data et commenter les 4 lignes qui le précède

In [1]:
import movielensutils as mlu
import scipy.sparse.linalg as la     # pour l'initialisation de Q0 ou P0
import numpy as np
from time import time 
import requests, zipfile, io

#Importing the data file
#takes some time to import the dataset
zip_url = "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
zip_file = requests.get(zip_url)
zip_file = zipfile.ZipFile(io.BytesIO(zip_file.content))
zip_file.extractall()

file_path = "ml-100k/u.data"

R, mask = mlu.load_movielens(file_path, minidata=False)
np.shape(R)


(943, 1682)

In [2]:
R1, mask1 = mlu.load_movielens(file_path, minidata=True)
np.shape(R1)

(100, 200)

Si minidata = True ; On préfère prendre une matrice de données de dimension réduite à 100x200.

### Question 1.2
Les données contiennent:
    - 943 utilisateurs
    - 1682 utilisateurs
    - 100000 notes disponibles

In [3]:
print(np.shape(R))
print(np.shape(mask[mask == True]))

(943, 1682)
(100000,)


###### Question 1.3
**1.3.1.  Convexité ** 

$f(P,Q) = \frac{1}{2} ||\mathbb{1}_{K}\circ(R-QP)||_{F}^2 + \frac{\rho}{2}||Q||_{F}^2+ \frac{\rho}{2}||P||_{F}^2 $


Au lieu d'étudier la convéxité de la fonction f(P,Q) pour P et Q deux matrices, on réduira le problème à l'étude de la fonction 
$f(p,q) = \frac{1}{2} ||\mathbb{\alpha}_{K}\circ(r-qp)||_{F}^2 + \frac{\rho}{2}||q||_{F}^2+ \frac{\rho}{2}||p||_{F}^2 $
où r,p et q sont des réels.

$f(p,q) = \frac{1}{2} ||\mathbb{\alpha}_{K}\circ(r-qp)||_{F}^2 + \frac{\rho}{2}||q||_{F}^2+ \frac{\rho}{2}||p||_{F}^2 $
$\quad = \frac{1}{2} (r-qp)^2 + \frac{\rho}{2}q^2+ \frac{\rho}{2}p^2 $
$\quad = \frac{1}{2} r^2 + \frac{1}{2} q^2p^2  - rqp + \frac{\rho}{2}q^2+ \frac{\rho}{2}p^2 $

$\nabla f(p,q) = [\quad q²p + \rho p - rq\quad ,\quad p²q - rq +\rho q \quad]^T$

$Hess f(p,q) =
  \begin{bmatrix}
    q²+\rho & 2pq-r \\
    2pq-r & p^2+\rho
  \end{bmatrix}
$

Pour p=q=r; $\quad det(Hess f(0,0)) = det( \begin{bmatrix} \rho & -r \\ -r & \rho \end{bmatrix} )  = \rho^2-r²$

$Si \quad r^2 > \rho^2, \qquad det(Hess f(0,0)) < 0 \quad \Rightarrow \quad$ La hessienne ne peut avoir que des valeurs propres positives.

$\Rightarrow \quad Hess f(0,0)$ ne peut être définie positive dans ce cas

Donc, $f(p,q)$ n'est pas convexe

Finalement, $f(P,Q)$ n'est pas convexe


**1.3.2.  Calcul du gradient de la fonction f**


$f(P,Q)=\frac{1}{2}\sum\limits_{(u,i)\in K} (R_{u,i}-\sum\limits_{c \in C} Q_{u,c}P_{c,i})^2 + \frac{\rho}{2}\sum\limits_{u\in U, c\in C} Q_{u,c}^2 + \frac{\rho}{2} \sum\limits_{i\in I, c\in C} P_{c,i}^2$

Soient:

$f_{1}(P,Q)= \frac{1}{2}\sum\limits_{(u,i)\in K} (R_{u,i}-\sum\limits_{c \in C} Q_{u,c}P_{c,i})^2 $

$f_{2}(P,Q)= \frac{\rho}{2}\sum\limits_{u\in U, c\in C} Q_{u,c}^2 $

$f_{3}(P,Q)= \frac{\rho}{2} \sum\limits_{i\in I, c\in C} P_{c,i}^2 $

$\bullet \frac{\partial f_{1}}{\partial P_{l,m}} \quad = \sum\limits_{i \in I, u \in U} (\mathbb{1}_{K})_{i,u} 	\circ (R_{i,u}-\sum\limits_{c \in C} Q_{i,c}P_{c,i})(-\sum\limits_{c \in C} Q_{i,c} \delta_{l,c} \delta_{m,u})$

$ \quad \quad \quad  = \sum\limits_{i \in I} (\mathbb{1}_{K})_{i,m} 	\circ (R_{i,m}-\sum\limits_{c \in C} Q_{i,c}P_{c,m})(- Q_{i,l})$

$\quad \quad  \quad = - \sum\limits_{i \in I} (\mathbb{1}_{K})_{i,m} 	\circ (R_{i,m}- (QP)_{i,m})Q_{i,l} $

$\frac{\partial f_{1}}{\partial P_{l,m}} \quad = - ( Q^{t} ( \mathbb{1}_{K} 	\circ (R - QP)) )_{l,m}$

$\bullet \frac{\partial f_{2}}{\partial P_{l,m}} \quad = 0 $

$ \bullet \frac{\partial f_{3}}{\partial P_{l,m}} \quad = \rho P_{l,m} $

$\Rightarrow $
$\nabla_{p} f (P,Q)  = -  Q^{t} ( \mathbb{1}_{K} 	\circ (R - QP))  + \rho P $

$\bullet \frac{\partial f_{2}}{\partial Q_{l,m}} \quad = \rho P_{l,m} $

$ \bullet \frac{\partial f_{3}}{\partial Q_{l,m}} \quad = 0 $

$f_{1} (P, Q+H) + f_{1} (P,Q) = \frac {1}{2} || \mathbb{1}_{K} 	\circ (R - (Q + H)P)||_{F} ^ 2 - \frac {1}{2} || \mathbb{1}_{K} 	\circ (R - QP)||_{F} ^ 2$
$ \quad \quad  \quad  \quad  \quad \quad \quad \quad \quad = \frac {1}{2} || \mathbb{1}_{K} 	\circ (R - QP)||_{F} ^ 2 - <\mathbb{1}_{K} 	\circ (R - QP),\mathbb{1}_{K} 	\circ HP> + \frac {1}{2} || \mathbb{1}_{K} 	\circ HP||_{F} ^ 2  -\frac {1}{2} || \mathbb{1}_{K} 	\circ (R - QP)||_{F} ^ 2  $

$ \quad \quad  \quad  \quad  \quad \quad \quad \quad \quad= - <\mathbb{1}_{K} 	\circ (R - QP),\mathbb{1}_{K} 	\circ HP> + o(H)$

$ \quad \quad  \quad  \quad  \quad \quad \quad \quad \quad = - <\mathbb{1}_{K} 	\circ (R - QP),HP> + o(H)$

$ \quad \quad  \quad  \quad  \quad \quad \quad \quad \quad= - tr ((\mathbb{1}_{K} 	\circ (R - QP))^{t} HP) + o(H)$

$ \quad \quad  \quad  \quad  \quad \quad \quad \quad \quad = - tr (P(\mathbb{1}_{K} 	\circ (R - QP))^{t} H) + o(H)$

$ \quad \quad  \quad  \quad  \quad \quad \quad \quad \quad = - <\mathbb{1}_{K} 	\circ (R - QP)P^{t},H> + o(H)$

$\Rightarrow $
$\nabla_{Q} f (P,Q)  = -  ( \mathbb{1}_{K} 	\circ (R - QP)) P^{t}  + \rho Q $


$\nabla f(P,Q) = [\nabla_{P} f (P,Q),\nabla_{Q} f (P,Q)]^T$

$\quad \quad \quad \quad= [-  Q^{t} ( \mathbb{1}_{K} 	\circ (R - QP))  + \rho P,-  ( \mathbb{1}_{K} 	\circ (R - QP)) P^{t}  + \rho Q]^T $

**1.3.3. Gradient Lipschitzien ? **                       

La lipschitziennité du gradient de f se traduit par l'existence d'un réel $L$ vérifiant $\forall (P1,Q1) \forall (P2,Q2)$ l'inégalité : 


$|| \nabla f(P1,Q1)- \nabla f(P1,Q1) ||_{F} \quad \le L ||\begin{pmatrix} 
 P1 - P2 \\ 
 Q1 - Q2  
 \end{pmatrix} ||_{F} $ 
 
 or, si on considère le cas scalaire, $\nabla f(p,q) = [\quad q²p + \rho p - rq\quad ,\quad p²q - rq +\rho q \quad]^T$
 
 On peut remarquer que le gradient s'écrit donc comme un polynome de degrès 3 en p et q et n'est donc pas majorable par un polynome de degrés 1 en p et q.
 
 $Hess f(p,q) =
  \begin{bmatrix}
    q²+\rho & 2pq-r \\
    2pq-r & p^2+\rho
  \end{bmatrix}
$

 $||Hess f(p,q)|| = \sqrt { (q²+\rho)^2 + (2pq-r)^2 + (2pq-r)^2 + (p^2+\rho)^2}$ n'est pas bornée
 
 en effet, tous les coefficients sont des polynômes en p et q qui ne sont pas bornés.

 On en déduit que $\nabla f$ n'est pas Lipschitzienne.

## 2. Trouver P quand $Q_0$ est fixé


### Question 2.1


$g(P) = \frac{1}{2} ||\mathbb{1}_{K}\circ(R-Q^0 P)||_{F}^2 + \frac{\rho}{2}||Q^0||_{F}^2+ \frac{\rho}{2}||P||_{F}^2 $

$P^{1} = argmin_{P} g(P)$

**2.1.1.  Convexité ** 

** convexité : **

$g(P) = f(P,Q_0) = \frac{1}{2} ||\mathbb{1}_{K}\circ(R-Q_0P)||^2 + \frac{\rho}{2}||Q_0||^2+ \frac{\rho}{2}||P||^2 $

$g(tP_1+(1-t)P_2) = \frac{1}{2} ||\mathbb{1}_{K}\circ(R-Q_0(tP_1+(1-t)P_2))||^2 + \frac{\rho}{2}||Q_0||^2+ \frac{\rho}{2}||tP_1+(1-t)P_2||^2$

$\quad = \frac{1}{2} ||\mathbb{1}_{K}\circ((tR+(1-t)R)-Q_0(tP_1+(1-t)P_2))||^2 + \frac{\rho}{2}||Q_0||^2+ \frac{\rho}{2}||tP_1+(1-t)P_2||^2$

$\quad = \frac{1}{2} ||(\mathbb{1}_{K}\circ(tR-tQ_0P_1))+(\mathbb{1}_{K}\circ((1-t)R-(1-t)Q_0P_2))||^2 + \frac{\rho}{2}||Q_0||^2+ \frac{\rho}{2}||tP_1+(1-t)P_2||^2$

$\quad \le \frac{1}{2} ||\mathbb{1}_{K}\circ(tR-tQ_0P_1)||^2 + \frac{1}{2} ||\mathbb{1}_{K}\circ((1-t)R-(1-t)Q_0P_2)||^2 + \frac{\rho}{2}||Q_0||^2+ \frac{\rho}{2}||tP_1||^2 + \frac{\rho}{2}||(1-t)P_2||^2$

$\quad \le \frac{1}{2}t||\mathbb{1}_{K}\circ(R-Q_0P_1)||^2 + \frac{1}{2} (1-t)||\mathbb{1}_{K}\circ(R-Q_0P_2)||^2 + \frac{\rho}{2}||Q_0||^2 + \frac{\rho}{2}t||P_1||^2 + \frac{\rho}{2}(1-t)||P_2||^2$

$\quad \le tg(P_1)+(1-t)g(P_2) $

Donc, g(P) est convexe.


**2.1.1.  Gradient de la fonction g ** 

Le gradient de g est celui de f quand Q est fixé à $Q^{0}$: 


$\nabla g(P) = -  (Q^0 )^{t} ( \mathbb{1}_{K} 	\circ (R - Q^0 P))  + \rho P $

In [4]:
R
rho = 0.3
#c = 4
Q0,sing,P0 = la.svds(R,k=4)
del sing
L0 = rho + np.sum(((np.transpose(Q0)).dot(Q0))**2)

### Question 2.2
Voir le code de la fonction objective dans movielensutils.py

In [5]:
# Requires the objective function defined in movielensutils.py

mask
R
Q0
P0
print('P0')
print(P0)
print('Q0')
print(Q0)
val0, grad_P0 = mlu.objective(P0, Q0, R, mask, 0.3)
print('grad_P0')
print(grad_P0)
print('g(P0)= %f' % val0)

P0
[[  1.62060750e-02   3.23391424e-03   4.88005961e-02 ...,  -9.53260775e-04
    7.73342642e-05   1.74971250e-03]
 [ -1.69737618e-02  -6.25039193e-02  -1.16405039e-02 ...,   5.33024145e-04
   -4.54336533e-04  -2.61400068e-04]
 [ -8.72397853e-02  -7.02505798e-03  -2.86181725e-02 ...,  -4.48134760e-04
    1.05231342e-04   2.03151884e-04]
 [  9.59509371e-02   3.51795155e-02   1.99288117e-02 ...,   3.03747116e-05
    3.31055915e-04   3.16852950e-04]]
Q0
[[ 0.08434724 -0.00613256  0.00597506  0.06580431]
 [-0.01628247  0.05257856 -0.04662602  0.01402104]
 [-0.02856416  0.02336183 -0.02561845  0.00565798]
 ..., 
 [ 0.01406097  0.00616532 -0.02502129  0.00744452]
 [-0.07353537  0.02288736  0.00809611  0.02403119]
 [ 0.03503116 -0.05854604 -0.01092715  0.04224209]]
grad_P0
[[ -2.55511347e+00  -5.13441086e-01  -7.73878335e+00 ...,   1.51419184e-01
   -1.22846636e-02  -2.77930288e-01]
 [  3.66873174e+00   1.35718942e+01   2.52859733e+00 ...,  -1.15949951e-01
    9.88378905e-02   5.68629500e-02]

** check_grad** : pour vérifier le calcul du gradient de g

In [6]:
#verifies the calculated value of the gradient at P0
#takes time to run

from numpy import ravel
from numpy import reshape

def func_g(vectP):
    P = reshape(vectP, (4, 1682))
    return mlu.objective(P, Q0, R, mask, 0.3)[0]
def grad_g(vectP):
    P = reshape(vectP, (4, 1682))
    return ravel(mlu.objective(P, Q0, R, mask, 0.3)[1])

from scipy.optimize import check_grad
erreur = check_grad(func_g,grad_g,ravel(P0));
print("erreur : %f " % erreur)
print("erreur relative: %f " % (erreur / np.linalg.norm(mlu.objective(P0, Q0, R, mask, rho)[1]) ) ) 

erreur : 1.117619 
erreur relative: 0.001520 


Le dernier résultat nous permet de confirmer le calcul et l'implémentation de $\nabla g(P)$

### Question 2.3

L'algorithme du gradient est basé sur: 
- $P \leftarrow P_0$
- faire : 
 - $\quad P_{k+1} \leftarrow P_k - \gamma\nabla g (P_k) \quad$ où $\gamma$ est le pas ici pris constant
- jusqu'à ce que $||\nabla g(P_k )||^2_F ≤ \epsilon$

In [7]:
Q0
rho
R
mask
def gradient(g, P0, gamma, epsilon):
    iterations = 0
    P = P0
    val,grad_P = g(P, Q0, R, mask, rho)
    while True:
        iterations = iterations + 1
        P = P - gamma*grad_P 
        val,grad_P = g(P, Q0, R, mask, rho)
        if ( ( (np.sum(grad_P ** 2))**(0.5) ) <= epsilon):
            break
    return P,val,iterations

### Question 2.4

In [8]:
epsilon = 1
P0
L0
g = mlu.objective #for now g is the defined objective function g(P)
start = time()
P_opt, val_opt, it = gradient(g,P0, 1/L0, 1) #L0 is the calculated Lipschitz constant
Runtime_gradient_pas_fixe = (time() - start) * 1000 # runtime in millis

print("Nombre d'itérations effectuées : %d" % it)
print("Le temps de calcul pour l'algorithme du gradient à pas fixe: \n %fms " % Runtime_gradient_pas_fixe)
print("La valeur optimale obtenue avec l'algorithme du gradient à pas fixe: \n %f" % val_opt)
#print('la matrice P qui donne cette valeur optimale est: ' )
#print(P_opt)

Nombre d'itérations effectuées : 50
Le temps de calcul pour l'algorithme du gradient à pas fixe: 
 854.984045ms 
La valeur optimale obtenue avec l'algorithme du gradient à pas fixe: 
 369551.700151


## 3. Raffinements algorithmiques pour le problème à $Q_0$ fixé

### Question 3.1
** Gradient à Pas Optimal: Armijo's Line Search **

Parmi les faiblesses de la méthode de gradient à pas fixe, on note la determination de la constante de Lipschitz elle même et que même si la fonction a un gradient Lipschitzien; l'estimation de la constante de Lipschitz peut prendre en compte les régions où la courbure est grande mais qui ne sont jamais atteintes par l'algorithme. Appliquer une méthode de recherche linéaire peut donc résoudre ce problème et donc choisir un pas $\gamma_{k} $  plus adapté en utilisant des informations locales.

L'idée de la méthode Linaire de Armijo est, étant donnée a ∈ (0, 1),b > 0 et β ∈ (0, 1), on veut determiner le meilleur entier **l** vérifiant l'inégalité suivante: 

$\quad f(x^+ (ba^l)) ≤ f(x_{k}) + β <∇f(x_{k}), x^+ ba^l − x_{k}> $

avec: $\gamma_{k} =  ba^l $

et $ \quad x^+ (\gamma_{k}) = x^+ (ba^l) = x_{k} - \gamma_{k} \nabla f (x_{k})$ 

ici on prend f = g et x= P, on aura alors: 

$\quad g(P^+ (ba^l)) ≤ g(x_{k}) + β <∇g(P_{k}), P^+ ba^l − P_{k}> $

$\quad \quad  \quad  \quad \quad  \leq  g(x_{k})  + β <∇g(P_{k}), P_ {k} - ba^l ∇g(P_{k}) − P_{k}>  $

$\quad \quad  \quad  \quad \quad  \leq  g(x_{k})  + β <∇g(P_{k}), - ba^l ∇g(P_{k}) >  $

$\quad \quad  \quad  \quad \quad  \leq  g(x_{k})  + β tr[(∇g(P_{k}))^{t} (P_ {k} - ba^l ∇g(P_{k}) − P_{k})] $

Pour cela, on designe par **g_Pplus** le terme à gauche de l'inégalité donné par : $ g_{P} plus = g(P^+ (ba^l)) ≤ g(x_{k}) + β $

et par ** ps**  le produit scalaire tel que: $ps =  <∇g(P_{k}), - ba^l ∇g(P_{k}) > $


In [9]:
Q0
rho
R
mask
def gradient_armijo(g, P0, epsilon):
    def armijo(a,b,beta,Pk):
        l = 0
        while True:
            g_Pk, grad_Pk = g(Pk, Q0, R, mask, rho)
            g_Pplus = g(Pk-b*(a**l)*grad_Pk , Q0, R, mask, rho)[0]
            ps = np.trace((np.transpose(grad_Pk)).dot(-b*(a**l)*grad_Pk))
            if ( g_Pplus <= ( g_Pk+beta*ps ) ):
                break
            l = l+1    
        return b*(a**l) #returns the determined gamma 
    
    P = P0
    val,grad_P = g(P, Q0, R, mask, rho)
    a = 0.5
    b = 1
    beta = 0.5
    it = 0
    while True:
        gamma = armijo(a,b,beta,P)
        P = P - gamma*grad_P 
        val,grad_P = g(P, Q0, R, mask, rho)
        it = it +1
        if np.sum(grad_P ** 2) <= epsilon:
            break
        b = 2 * gamma
    return P,val,it 

g
start = time()
P_armijo, val_armijo,it = gradient_armijo(g,P0, 1)
Runtime_gradient_armijo = (time() - start) * 1000 # time in millis

print("Nombre d'itérations effectuées : %d" % it)
print("Temps d'execution de l'algorithme avec recherche linéaire: \n %f ms" % Runtime_gradient_armijo)
print("valeur retrouvé par l'algorithme avec recherche linéaire: \n %f" % val_armijo)
#print('P_opt_armijo= ' )
#print(P_armijo)

Nombre d'itérations effectuées : 7
Temps d'execution de l'algorithme avec recherche linéaire: 
 557.163000 ms
valeur retrouvé par l'algorithme avec recherche linéaire: 
 369551.060504


### Question 3.2

**Gradient Conjugué (Cas d'une fonction quadratique) **

En se référant au paragraphe III.8.1 du cours de MDI210, On pourrait appliquer la méthode du gradient conjugué pour une fonction quadratique dont la matrice est symetrique, définie, positive.

$ g(P) = \frac{1}{2}\left|\left|
\mathbb{1}_K \circ (R - Q_0P) \right|\right|_F^2 + \frac{\rho}{2}\left|\left|
Q_0\right|\right|_F^2 + \frac{\rho}{2}\|
P \|_F^2 = \frac{1}{2} \left( \left(\mathbb{1}_K \circ (R - Q_0P)\right)^T\mathbb{1}_K \circ(R - Q_0P)\right) + \frac{\rho}{2}\left( {Q_0}^TQ_0 \right) + \frac{\rho}{2}\left( P^TP \right) = \frac{1}{2} P^T \left( {Q_0}^T \mathbb{1}_K \mathbb{1}_K^T {Q_0} + \rho I\right) P - \frac{1}{2} \left( P^T{Q_0}^T(\mathbb{1}_K \circ R) + (\mathbb{1}_K \circ R)^TQP \right) + \frac{1}{2}\left( (\mathbb{1}_K \circ R)^T(\mathbb{1}_K \circ R) + \rho {Q_0}^TQ_0\right)$

$\Rightarrow \quad g(P) = \frac{1}{2}P^TAP + b^TP + P^Tb + c$ 

et
$$\begin{align*} 
A &= {Q_0}^T (\mathbb{1}_K \mathbb{1}_K^T){Q_0} + \rho I \\ 
b &= -\frac{1}{2} {Q_0}^T(\mathbb{1}_K \circ R)
\\ c &= \frac{1}{2}\left( (\mathbb{1}_K \circ R)^T(\mathbb{1}_K \circ R) + \rho {Q_0}^TQ_0\right)
\end{align*}$$

A est bien une matrice symetrique, definie, positive. 

Donc, on peut utiliser l'algorithme du gradient conjugué.

**Gradient Conjugué (Cas d'une fonction quelconque) **
 
Pour s'affranchir de la determination des paramètres de cette fonction quadratique, nous proposons d'utiliser la méthode du gradient conjugué pour une fonction quelconque, présentée notamment dans le cours de MDI210 paragraphe III.8.2.
Le principe de l'algorithme est le suivant:

- partir d'un point $P_0$ ;
- faire $\quad d_0 \leftarrow −\nabla g_(P_0 ) \quad et \quad k \leftarrow 0$ ;
- répéter
 - choisir $s_k$ minimisant $g(P_k + sd_k )$, par rapport à s
 - $P_{k+1} \leftarrow P_k + s_kd_k$
 - $b_k \leftarrow \frac{||\nabla g(P_{k+1})||^2}{||\nabla g(P_{k})||^2}$
 - $d_{k+1} \leftarrow −\nabla g(P_{k+1}) + b_kd_k$
 - $k \leftarrow k+1$
- jusqu'à ce qu'un test d'arrêt soit vérifié. 

Ici, On prendra comme condition d'arrêt: $||\nabla g(P_k)|| \le \epsilon$

L'autre difficulté dans l'algorithme est la determination de s qui minimise $g(P_k + sd_k )$. Pour cela on considérera l'expression analytique de $g(P_k + sd_k )$

$g(P_k + sd_k) $

$\quad = \frac{1}{2} ||\mathbb{1}_{K}\circ(R-(Q_k+sd_k)P_0)||^2 + \frac{\rho}{2}||Q_k+sd_k||^2+ \frac{\rho}{2}||P_0||^2$

$\quad = \frac{1}{2} ||\mathbb{1}_{K}\circ(R-(Q_kP_0)||^2 + \frac{1}{2} ||\mathbb{1}_{K}\circ(sd_kP_0)||^2 - <\mathbb{1}_{K}\circ(R-(Q_kP_0), \mathbb{1}_{K}\circ(sd_kP_0)> + \frac{\rho}{2}||Q_k||^2 + \frac{\rho}{2}||sd_k||^2 + \rho <Q_k, sd_k> + \frac{\rho}{2}||P_0||^2$

$\quad = \frac{1}{2} ||\mathbb{1}_{K}\circ(R-(Q_kP_0)||^2 + \frac{1}{2} s^2||\mathbb{1}_{K}\circ(d_kP_0)||^2 - s<\mathbb{1}_{K}\circ(R-(Q_kP_0), \mathbb{1}_{K}\circ(d_kP_0)> + \frac{\rho}{2}||Q_k||^2 + \frac{\rho}{2}s^2||d_k||^2 + \rho <Q_k, sd_k> + \frac{\rho}{2}||P_0||^2$

$\frac{\partial g(P_k + sd_k)}{\partial s} = s||\mathbb{1}_{K}\circ(d_kP_0)||^2 - <\mathbb{1}_{K}\circ(R-(Q_kP_0), \mathbb{1}_{K}\circ(d_kP_0)> + \rho s||d_k||^2 + \rho <Q_k, d_k> $

En écrivant $\frac{\partial g(P_k + sd_k)}{\partial s} = 0$, On retrouve:

$s_{min} = \frac{<\mathbb{1}_{K}\circ(R-(Q_kP_0), \mathbb{1}_{K}\circ(d_kP_0)> - \rho <Q_k, d_k>}{||\mathbb{1}_{K}\circ(d_kP_0)||^2 + \rho ||d_k||^2}$



In [10]:
def scal(A,B):
    return ( np.trace((np.transpose(A)).dot(B)) )

def s_min(Pk, dk, Q0, R, mask, rho ):
    num = -rho*scal(Pk,dk) - scal( (mask*(Q0.dot(Pk)-R) ) , (mask*(Q0.dot(dk)) ) )
    denom = rho * scal(dk,dk) + scal(mask*(Q0.dot(dk)),mask*(Q0.dot(dk)) )
    s = num / denom
    return s

epsilon = 1
R
mask
rho

def gradient_conjugue (g,P0, Q0, R, mask, rho, epsilon):
    Q0    
    P0
    dk =  -g(P0, Q0, R, mask, rho)[1] #initialisation 
    k = 0
    Pk = P0
    iterations = 0
    while True:
        iterations = iterations + 1
        s = s_min(Pk, dk, Q0, R, mask, rho)
        Pk1 = Pk + s*dk
        grad_Pk = g(Pk, Q0, R, mask, rho)[1]
        grad_Pk1 = g(Pk1, Q0, R, mask, rho)[1]
        bk = scal(grad_Pk1,grad_Pk1)/ scal(grad_Pk,grad_Pk)
        dk = -grad_Pk1 + bk*dk # mise à jour de dk1
        Pk = Pk1
        if (  np.sum(grad_Pk1 ** 2) <= epsilon ):
            break
    return Pk, iterations

start = time()
Pkmin, it = gradient_conjugue (g,P0, Q0, R, mask, rho, epsilon)
Runtime_gradient_conjugue = (time() - start) * 1000 # time in millis

print("Nombre d'itérations effectuées : %d" % it)
print("Le temps d'execution de l'algorithme du gradient conjugué : \n %fms" % Runtime_gradient_conjugue)
print("valeur optimale par l'algorithme du gradient conjugué : \n %f" % g(Pkmin, Q0, R, mask, rho)[0])
#print("P_opt_gradientConjugué : ")
#print(Pkmin)

Nombre d'itérations effectuées : 6
Le temps d'execution de l'algorithme du gradient conjugué : 
 2339.318037ms
valeur optimale par l'algorithme du gradient conjugué : 
 369550.880233


### Question 3.3

En vue de comparer la rapidité des algorithmes, on peut appliquer l'un des deux critères suivants:

 - **Critère 1**: **Runtime**
 
On utilise la fonction time.time() pour estimer les temps de calculs des algorithmes implémentés.

Les **Runtime** de chacuns des trois algorithmes sont comme suit: 
 
   **$\diamond $ Gradient à pas constant**:  839.606047ms
   
   **$\diamond $ Méthode de Recherche Linéaire de Armijo**:  539.134026 ms
   
   **$\diamond $ Gradient Conjugué **: 2165.952921ms

$\Rightarrow $ Donc on peut dire  que la méthode de recherche linéaire de Armijo est la plus rapide car elle nous permet d'eviter les pas trés grands qui augmentent le nombre d'itérations comme cet algorithme consiste à reduire progressivement la valeur du paramètre d'accélération jusqu'à en trouver une qui satisfait la condition d'Armijo.
L'algorithme du gradient conjugué est le plus lent cela peut être dû à l'implémentation choisie qui n'utilise pas le fait que g est quadratique et qui implémente un algorithme générique faisant intervenir des calculs supplémentaires. 
 
 - **Critère 2**: **Comptage des itérations **

Les **nombres d'itérations ** de chacuns des trois algorithmes sont comme suit: 
 
   **$\diamond $ Gradient à pas constant**:  50
   
   **$\diamond $ Méthode de Recherche Linéaire de Armijo**:  7
   
   **$\diamond $ Gradient Conjugué **: 6

$\Rightarrow $ Contrairement au classement fournit par le critère précédent, on voit qu'ici c'est la méthode du gradient conjugué qui offre une convergence plus rapide et proche à la recherche linéaire. Cependant, Le gradient à pas constant nécessite un grand nombre d'itérations ce qui est expliqué par le fait même que le pas soit constant et que l'algorithme doit tourner plusieurs fois pour avancer dans une direction données contrairement au deux autres algorithmes qui optimise la longueur du pas effectué.

## 4.Résolution du problème complet
### Question 4.1

L'algorithme prend beaucoup de temps pour tourner. C'est pour cela qu'il y a deux alternative dans le code.
La première alternative "alternative 1" n'est pas précise mais ne prend qu'environ 90 secondes.
Si vous avez le temps commentez l' "alternative 1" et décommenter l' "alternative 2"

In [11]:
rho
R
mask

def gradientPQ(g, P0, Q0, epsilon):
    
    def armijoPQ(a,b,beta,Pk,Qk):
        l = 0
        g_k, grad_Pk, grad_Qk = g(Pk, Qk, R, mask, rho)
        while True:
            Pplus = Pk-b*(a**l)*grad_Pk
            Qplus = Qk-b*(a**l)*grad_Qk
            g_plus, grad_Pplus, grad_Qplus = g(Pplus ,Qplus , R, mask, rho)
            
            ps = np.trace(np.concatenate((grad_Pk,np.transpose(grad_Qk)),axis=1).dot(np.transpose(np.concatenate((Pplus-Pk,np.transpose(Qplus-Qk)),axis=1))))
            if ( g_plus <= ( g_k+beta*ps ) ):
                break
            l = l+1    
        return b*(a**l) #returns the determined gamma 
    
    P = P0
    Q = Q0
    g = mlu.total_objective
    val,grad_P,grad_Q = g(P, Q, R, mask, rho)
    a = 0.5
    b = 1
    beta = 0.5
    i = 0
    while True:
        gamma = armijoPQ(a,b,beta,P,Q)
        P = P - gamma*grad_P
        Q = Q - gamma*grad_Q
        val,grad_P,grad_Q = g(P, Q, R, mask, rho)
        b = 2 * gamma
        
        #YOU CAN UNCOMMENT the "alternative 2" line if you have time to wait for the code to terminate BUT
        #IF YOU DO, then DON'T forget to comment the "alternative 1" and to restore the code after you've finished
        if  ( ( np.sum(grad_P ** 2) ) < epsilon ): #alternative 1
        #if ( np.sum(grad_P ** 2) + np.sum(grad_Q ** 2) ) < epsilon: #alternative 2
            break
           
    return P, Q, val 
    
epsilon = 100
g=mlu.total_objective
start = time()
P_armijo, Q_armijo, val_armijo = gradientPQ(g,P0,Q0, epsilon)
Runtime_gradientCroise = time()-start

print("Le temps d'execution est : %f " % Runtime_gradientCroise)
print("La valeur retrouvé est : \n %f" % val_armijo)

print("Les solutions retrouvées sont :")
print("P: ")
print(P_armijo)
print("Q: ")
print(Q_armijo)

Le temps d'execution est : 89.285617 
La valeur retrouvé est : 
 34365.408888
Les solutions retrouvées sont :
P: 
[[-0.26905088 -0.10910263  0.6979702  ...,  0.03228779 -0.20605116
   0.30560488]
 [-0.1072123  -0.46977553 -0.19797729 ...,  0.39087079  0.01151933
   0.41791422]
 [-0.19639943 -0.20853861 -0.84937913 ..., -0.62263778  0.25613586
   0.08047297]
 [ 1.93876026  1.60361052  1.34599701 ...,  0.26968794  1.39581177
   1.1991189 ]]
Q: 
[[ 0.54378289  0.57471236  0.09489071  2.16749607]
 [-0.40239689  0.67843883  0.46357677  2.15438001]
 [ 0.90727517 -0.46432211  0.5966598   2.1105597 ]
 ..., 
 [-0.03635137  0.73986335 -0.4244401   2.19406521]
 [-1.22354587 -0.06878356  0.03136357  2.13184373]
 [ 0.93484468 -0.29639925 -0.73780161  2.02853328]]


On est en train de minimiser la fonction f(P,Q) qui n'est pas convexe. Les minimums retrouvés sont, donc, susceptibles d'être des minimums locaux.

### Question 4.2

Montrons que la valeur de l’objectif décroit à chaque itération pour enfin déduire la convergence.

D'abord, l'algorithme de la méthode des moindres carrés alternés est:

$\quad$ Pour $k\ge1$ faire :

$\qquad P_k\leftarrow argmin_P f(P, Q_{k-1})$

$\qquad Q_k \leftarrow argmin_Q f(P_k, Q)$

$\quad$ fin

à l'étape k,

notons la valeur minimale retrouvée jusque là: $f_{k-1,2} = f(P_{k-1},Q_{k-1})$

La première partie de l'étape k est $\quad P_k \leftarrow arg min_P f(P, Q_{k-1}) $

Donc $f_{k,1} = f(P_k,Q_{k-1}) \le f(P_{k-1},Q_{k-1}) = f_{k-1,2}$

La deuxième partie est $\quad Q_k \leftarrow argmin_Q f(P_k, Q)$

Donc $f_{k,2} = f(P_k,Q_k) \le f(P_k,Q_{k-1}) = f_{k,1}$

Finalement $f_{k,2} \le f_{k-1,2}$

On remarque bien que la valeur de la fonction objective décroît à chaque nouvelle itération.

Si on note $v_k = f_{k,2}$ la valeur de la fonction objective retournée à l'étape k de l'algorithme, On remarque que $v_k$ est une suite décroissante

De plus, $v_k$ est bornée car $f(P^*,Q^*)=min_{(P,Q)} f(P,Q) \le v_k$ et de plus elle prend des valeurs dans $\mathbb{R_+}$ donc minorée par 0 au pire des cas.

Donc $v_k$ converge vers $f(P^*,Q^*)$

### Question 4.3

Implémentation de la méthode des moindres carrés alternés avec l'algorithme de recherche linéaire pour la minimisation par rapport à P et Q.

L'***algorithme prend beaucoup de temps pour tourner***. Si vous n'avez pas assez de temps (~25 minutes), commentez la boucle while et utiliser la boucle for (la boucle for prend environ 230secondes) qui est en commentaire. Rétablissez le code quand vous aurez fini.

In [20]:
g = mlu.total_objective 

def min_P_armijo(g, P0, Q0, epsilon):
    def armijoP(a,b,beta,Pk):
        l = 0
        while True:
            g_Pk, grad_Pk = g(Pk, Q0, R, mask, rho)[0:2]
            g_Pplus = g(Pk-b*(a**l)*grad_Pk , Q0, R, mask, rho)[0]
            ps = np.trace((np.transpose(grad_Pk)).dot(-b*(a**l)*grad_Pk))
            if ( g_Pplus <= ( g_Pk+beta*ps ) ):
                break
            l = l+1    
        return b*(a**l) #returns the determined gamma 
    
    P = P0
    val, grad_P = g(P, Q0, R, mask, rho)[0:2]
    a = 0.5
    b = 1
    beta = 0.5
    while True:
        gamma = armijoP(a,b,beta,P)
        P = P - gamma*grad_P 
        val,grad_P = g(P, Q0, R, mask, rho)[0:2]
        if np.sum(grad_P ** 2) <= epsilon:
            break
        b = 2 * gamma
    return P,val

def min_Q_armijo(g, P0, Q0, epsilon):
    def armijoQ(a,b,beta,Qk):
        l = 0
        while True:
            g_Qk, grad_Pk, grad_Qk = g(P0, Qk, R, mask, rho)
            del grad_Pk
            g_Qplus = g(P0, Qk-b*(a**l)*grad_Qk , R, mask, rho)[0]
            ps = np.trace((np.transpose(grad_Qk)).dot(-b*(a**l)*grad_Qk))
            if ( g_Qplus <= ( g_Qk+beta*ps ) ):
                break
            l = l+1    
        return b*(a**l) #returns the determined gamma 
    
    P0
    Q = Q0
    val,grad_P,grad_Q = g(P0, Q, R, mask, rho)
    a = 0.5
    b = 1
    beta = 0.5
    while True:
        gamma = armijoQ(a,b,beta,Q)
        Q = Q - gamma*grad_Q 
        val,grad_P, grad_Q = g(P0, Q, R, mask, rho)
        if np.sum(grad_Q ** 2) <= epsilon:
            break
        b = 2 * gamma
    return Q,val

pmin = P0
qmin= Q0
start = time()

#for i in range(0,200): #UNCOMMENT THIS AND COMMENT NEXT LINE, if you can't wait
it = 0
while True: 
    it = it +1
    pmin = min_P_armijo(g,pmin,qmin,epsilon)[0]
    qmin = min_Q_armijo(g,pmin,qmin,10**7)[0] #minimizing over q is very time consuming
    val, gradP, gradQ = mlu.total_objective(pmin, qmin, R, mask, rho)
    if np.sum(gradQ ** 2) <= epsilon:
        if np.sum(gradP ** 2) <= epsilon:
            print("Algorithm Terminated Successfully")
            break;
            
    #Cela permet l'affichage des valeurs chaque 40 itérations pour vérifier la décroissance
    if ( (it%40) == 0 ):
        print(val)
            
runtime_alterne_armijo = time()- start
        
print("Le temps d'execution est : %f" % runtime_alterne_armijo)
print("La valeur optimale est : %f " % g(pmin, qmin, R, mask, rho)[0])
print("Les solutions retrouvées sont :")
print("P: ")
print(pmin)
print("Q: ")
print(qmin)

60927.3633929
51256.4785339
46635.8361022
44186.812017
42647.7271689
41362.7454045
40343.4487067
39690.6186819
39075.1833426
38619.5663492
38237.1690922
37903.5865658
37547.3136085
37239.9941206
37001.3218913
36733.4965158
36551.2558392
36364.2261847
36206.5137283
36086.0431964
35965.4531846
35871.7257751
35781.7972493
35699.4094447
35622.42491
35541.3314129
35468.6098347
35400.2080874
35332.8389469
35265.8504874
35201.3643112
35147.1952648
35101.3491284
35050.0781417
35002.9657858
34959.0587944
34911.0091055
34878.2262654
34854.7318397
34824.257885
34793.4732999
34767.8710286
34743.7174995
34719.1510414
34698.1714615
34678.1938744
34660.8368751
34644.7732738
34629.8969562
34614.0925442
34601.1545683
34587.9235674
34575.98204
34565.1391949
34554.4616832
34544.4699103
34534.974726
34525.4315594
34516.7862856
34508.8201799
34500.0539486
34492.9253111
34485.863383
34479.2763564
34472.2829565
34465.8995734
34459.4862142
34453.0100197
34446.8601779
34441.4024715
34435.9040164
34430.3487474


### Question 4.4

comparaison entre les résultats:

La comparaison effectuée ici n'est pas pertinente car on a forcé dans l'implémentation un arrêt prématuré des algorithmes vu qu'ils prennent beaucoup de temps.

- ** Les solutions **


In [22]:
print("Les solutions retrouvées par la méthode du gradient avec recherche linéaire :")
print("P: ")
print(P_armijo)
print("Q: ")
print(Q_armijo)

print("Les solutions retrouvées par les moindres carrés alternés :")
print("P: ")
print(pmin)
print("Q: ")
print(qmin)

from math import sqrt
print("distance entre les P : %f " % sqrt(np.sum( (P_armijo-pmin)**2 )) )
print("distance entre les Q : %f " % sqrt(np.sum( (Q_armijo-qmin)**2 )) )

Les solutions retrouvées par la méthode du gradient avec recherche linéaire :
P: 
[[-0.26905088 -0.10910263  0.6979702  ...,  0.03228779 -0.20605116
   0.30560488]
 [-0.1072123  -0.46977553 -0.19797729 ...,  0.39087079  0.01151933
   0.41791422]
 [-0.19639943 -0.20853861 -0.84937913 ..., -0.62263778  0.25613586
   0.08047297]
 [ 1.93876026  1.60361052  1.34599701 ...,  0.26968794  1.39581177
   1.1991189 ]]
Q: 
[[ 0.54378289  0.57471236  0.09489071  2.16749607]
 [-0.40239689  0.67843883  0.46357677  2.15438001]
 [ 0.90727517 -0.46432211  0.5966598   2.1105597 ]
 ..., 
 [-0.03635137  0.73986335 -0.4244401   2.19406521]
 [-1.22354587 -0.06878356  0.03136357  2.13184373]
 [ 0.93484468 -0.29639925 -0.73780161  2.02853328]]
Les solutions retrouvées par les moindres carrés alternés :
P: 
[[-0.39813087 -0.25449876  0.88186214 ...,  0.25076266 -0.25796946
   0.29890929]
 [ 0.2322916  -0.23029654 -0.03849453 ...,  0.40216265  0.33405063
   0.73774794]
 [-0.07487866 -0.10068693 -1.10998596 ..., 

  - ** La prédiction R  = QP **

In [23]:
#la méthode du gradient avec recherche linéaire (4.1)
Rgrl = Q_armijo.dot(P_armijo)
print("la prédiction de R avec la méthode du gradient avec recherche linéaire")
print(Rgrl)

# la méthode des moindres carrés alternés
Rmca = qmin.dot(pmin) 
print("la prédiction de R avec les moindres carrés altérenés")
print(Rmca)

from math import sqrt
print("distance entre les prédictions R : %f " % sqrt(np.sum( (Rmca-Rgrl)**2 )) )


la prédiction de R avec la méthode du gradient avec recherche linéaire
[[ 3.97569727  3.12671719  3.1026093  ...,  0.76766082  2.94429466
   3.0130848 ]
 [ 4.1213084   3.08330141  2.0908601  ...,  0.5445593   3.21657713
   2.78121796]
 [ 3.7803635   3.3792302   3.05919292 ...,  0.04549351  2.90647627
   2.66204791]
 ..., 
 [ 4.26758415  3.26333454  3.14186747 ...,  1.14400263  2.96980069
   2.89487935]
 [ 4.46354467  3.57791143  2.00243475 ...,  0.48901332  3.23500661
   2.1561908 ]
 [ 3.85800039  3.44408453  4.06824706 ...,  0.92078436  2.44643304
   2.53490313]]
la prédiction de R avec les moindres carrés altérenés
[[ 3.97811857  3.16690008  2.88471756 ...,  1.17806033  3.17275393
   2.96009527]
 [ 4.01713116  3.04980852  2.07503587 ...,  1.02891881  3.19849715
   2.72926189]
 [ 3.42726248  3.01185901  2.98337263 ...,  0.89721168  2.71888894
   2.47055753]
 ..., 
 [ 4.4535405   3.55444876  2.88714012 ...,  1.22652534  3.40118565
   2.90874476]
 [ 4.48260302  3.62729485  1.88515871 ..

- ** La valeur de la fonction objective **

Ici, la valeur retrouvée par la méthode des moindres carrés alternés est moins bonne que celle retrouvée par la méthode du gradient avec recherche linéaire. Cela est dû au fait que l'implémentation executée force un arrêt de la méthode des moindres carrés alternés avec la convergence (en raison du grand temps de calcul).

In [24]:
#la méthode du gradient avec recherche linéaire 
print("La valeur retrouvé par le gradient avec recherche linéaire : \n %f" % val_armijo)

#la méthode des moindres carrés alternés
print("La valeur retrouvé par la méthode des moindres carrés alternés : \n %f" % g(pmin, qmin, R, mask, rho)[0])

La valeur retrouvé par le gradient avec recherche linéaire : 
 34365.408888
La valeur retrouvé par la méthode des moindres carrés alternés : 
 34405.434410


- ** Comparaison des temps de calcul **

La méthode des moindres carrés alternés prend plus de temps que la méthode du gradient avec recherche linéaire. Mais, la méthode des moindres carrés alternés fournit (potentiellement) une meilleure solution.

En effet:

In [25]:
print("Le temps d'execution du gradient avec recherche linéaire est : %f " % Runtime_gradientCroise)
print("Le temps d'execution des moindres carrés alternés est : %f" % runtime_alterne_armijo)

Le temps d'execution du gradient avec recherche linéaire est : 89.285617 
Le temps d'execution des moindres carrés alternés est : 1466.954957


### Question 4.5

- Quel ﬁlm recommanderiez-vous à l’utilisateur 300 ?

On définit **mask_br** afin d'éliminer les films dont l'utilisateur 300 connait et donc les scores associés. Par suite, on ne laisse que les films dont l'utilisateur ne connait pas le score. Apres on determine le film **best_predicted_item** correspondant au maximum du score **best_rate_predicted_item**. 

In [26]:
mask_br = np.full((943,1682),True)
mask_br[mask] = False
R_unseen = Rgrl*mask_br
User300= R_unseen[299,:]

In [27]:
best_rate_predicted_item = np.amax(User300)
print("Le meilleur score est : %f " % best_rate_predicted_item)

Le meilleur score est : 7.378320 


In [28]:
list300 = User300.tolist()
best_predicted_item = list300.index(best_rate_predicted_item)
print('Le film recommendé à l ''utilisateur 300 est : %d' % best_predicted_item)

Le film recommendé à l utilisateur 300 est : 1292
